In [1]:
# Import necessary libraries:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import warnings
import os

# Ignore harmless warnings:
warnings.filterwarnings('ignore')

# Apply df dispaly format for pandas:
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# Word Bank Data Converter:
def wb_data_panelizer(df: pd.DataFrame, target_years: tuple, target_col: str) -> pd.DataFrame:
    """Takes World Bank data excel file and converts it to desirable format"""
    
    # Remove irrelevant columns
    df = df.drop(columns=['Indicator Name', 'Indicator Code'])
    
    # Group by country:
    df = df.groupby(by=['Country Name', 'Country Code'], as_index=False).first()
    
    # Initialize output data:
    out_data = pd.DataFrame(data={'country': [], 'code': [],
                  "year": [], f'{target_col}': []})
    
    # Get values for each country and corresponding year:
    for idx in range(df.shape[0]):
        # Get country name and codes from index":
        country = df.iloc[idx]['Country Name']
        code = df.iloc[idx]['Country Code']
        
        # Get each year value from target years:
        for year in range(target_years[0], target_years[1] + 1):
            
            # Find corresponding year value:
            value = df.iloc[idx][str(year)]
            
            # Append to outdata:
            temp_data=pd.DataFrame(data={'country': country, 'code': code, "year": str(year), f'{target_col}': value}, index=[0])
            out_data = pd.concat([out_data, temp_data])
    
    # Return the "panelized" data:
    return out_data

In [3]:
# Initialize dataframes list:
dfs = []

# Find all wb datasets:
for root, dirs, files in os.walk('.'):
    
    # iterate all files in current directory:
    for file in files:
        if file.startswith('wb'):
            
            # Append the panelized dataframes to the dfs list:
            df = pd.read_excel(file, header=3)
            dfs.append(wb_data_panelizer(df=df, target_years=(1996, 2022), target_col=file))

In [4]:
# Initialize merger:
merged = pd.DataFrame(columns=['country','code','year'])

# Merge dfs based on Country, country code and year:
for df in dfs:
    merged = pd.merge(left=merged, right=df, how='outer', on=['country',
                                                              'code',
                                                              'year'])

In [5]:
merged.columns

Index(['country', 'code', 'year', 'wb_pop_growth.xls', 'wb_pop.xls',
       'wb_gross_capital_formation.xls', 'wb_per_capita_gpd_constant_2015.xls',
       'wb_trade_openness.xls', 'wb_rule_of_law_estimate.xls'],
      dtype='object')

In [6]:
# Fix column names:
new_cols = ['country', 'code', 'year',
            'n', 'pop', 'I', 'y', 'TRADE', 'INST']

# Change column names:
merged.columns = new_cols

In [7]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7182 entries, 0 to 7181
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7182 non-null   object 
 1   code     7182 non-null   object 
 2   year     7182 non-null   object 
 3   n        7154 non-null   float64
 4   pop      7155 non-null   float64
 5   I        5599 non-null   float64
 6   y        6725 non-null   float64
 7   TRADE    5827 non-null   float64
 8   INST     4873 non-null   float64
dtypes: float64(6), object(3)
memory usage: 505.1+ KB


In [8]:
# Read human capital data from pwt1001:
pwt = pd.read_excel('pwt1001.xlsx')
pwt.head()

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_x,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k
0,ABW,Aruba,Aruban Guilder,1950,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,Aruba,Aruban Guilder,1951,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABW,Aruba,Aruban Guilder,1952,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,Aruba,Aruban Guilder,1953,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABW,Aruba,Aruban Guilder,1954,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Limit pwt for 1996 and later:
pwt = pwt.query('year >= 1996')

In [10]:
# Define HC
pwt['HC'] = pwt['hc']

# Remove null countries:
to_remove = pwt[pwt['HC'].isna()]['countrycode'].unique()
pwt = pwt.query('countrycode not in @to_remove')

In [11]:
# Merge by country code and year:
pwt['code'] = pwt['countrycode'] # Fix the key matching
pwt['year'] = pwt['year'].astype(str) # Fix type for merge.

# Merge dfs:
merged = pd.merge(left=merged, right=pwt[['code', 'year','HC']],
                  on=['code', 'year'], how='left')

In [12]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7182 entries, 0 to 7181
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7182 non-null   object 
 1   code     7182 non-null   object 
 2   year     7182 non-null   object 
 3   n        7154 non-null   float64
 4   pop      7155 non-null   float64
 5   I        5599 non-null   float64
 6   y        6725 non-null   float64
 7   TRADE    5827 non-null   float64
 8   INST     4873 non-null   float64
 9   HC       3456 non-null   float64
dtypes: float64(7), object(3)
memory usage: 561.2+ KB


In [13]:
# These values below are disregarded in merge
print(set(pwt['code']) - set(merged['code']))
print(set(merged['code']) - set(pwt['code'])) # Here wb countries covered all

{'TWN'}
{'ASM', 'COM', 'MNA', 'DMA', 'SYC', 'TLS', 'VIR', 'TEA', 'CUW', 'ECS', 'LMC', 'LCN', 'SLB', 'EAR', 'PSE', 'KIR', 'PRI', 'GRD', 'CUB', 'PLW', 'TCD', 'CHI', 'AND', 'GEO', 'TSS', 'PRK', 'BIH', 'EAP', 'EAS', 'PYF', 'FRO', 'PST', 'SOM', 'MAF', 'VCT', 'TKM', 'UMC', 'SST', 'INX', 'TCA', 'DJI', 'LTE', 'SSF', 'SSD', 'TEC', 'MEA', 'IDA', 'CSS', 'WSM', 'AFW', 'LMY', 'MNP', 'TUV', 'ABW', 'AFE', 'ERI', 'NCL', 'STP', 'HIC', 'AFG', 'LIC', 'BTN', 'TLA', 'NRU', 'VGB', 'OED', 'GUM', 'LBN', 'GIB', 'IDB', 'HPC', 'VUT', 'CYM', 'GRL', 'NAC', 'OMN', 'TMN', 'PSS', 'LAC', 'SMR', 'PNG', 'EMU', 'MNE', 'EUU', 'CEB', 'IBD', 'GNQ', 'KNA', 'ARB', 'ECA', 'ATG', 'BHS', 'TON', 'SXM', 'PRE', 'AZE', 'IBT', 'FSM', 'GNB', 'LDC', 'LBY', 'SAS', 'BMU', 'GIN', 'SUR', 'MIC', 'UZB', 'LCA', 'FCS', 'LIE', 'XKX', 'CPV', 'MKD', 'MHL', 'OSS', 'IDX', 'TSA', 'BLR', 'MCO', 'WLD', 'SSA', 'IMN'}


In [14]:
# Merge total assets and liabilities data:
lane_and_ferretti = pd.read_excel('Lane_Milesi_Ferretti_Financial_data.xlsx')
lane_and_ferretti.head()

,Country,Year,Portfolio equity assets,Portfolio equity liabilities,FDI assets,FDI liabilities,Debt assets (portfolio debt + other investment),Debt liabilities (portfolio debt + other investment),financial derivatives (assets),financial derivatives (liabilities),...,Portfolio debt liabilities,Other investment assets,Other investment liabilities,Current account balance,Capital account,Exchange rate (period avg),Exchange rate (end of period),net IIP excl gold / GDP domestic currency,net IIP / GDP domestic currency,FINANCIAL
0,Afghanistan,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,433.5477,438.4334,NaN,NaN,NaN
1,Afghanistan,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,433.5477,438.4334,NaN,NaN,NaN
2,Afghanistan,1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,433.5477,438.4334,NaN,NaN,NaN
3,Afghanistan,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,433.5477,438.4334,NaN,NaN,NaN
4,Afghanistan,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,433.5477,438.4334,NaN,NaN,NaN


In [15]:
# Fix column names for merge:
lane_and_ferretti['country'] = lane_and_ferretti['Country']
lane_and_ferretti['year'] = lane_and_ferretti['Year']
lane_and_ferretti['year'] = lane_and_ferretti['year'].astype(str)

In [16]:
# Eliminate mismatches:
set(lane_and_ferretti['country']) - set(merged['country'])

{'Czech Republic',
 'Egypt',
 'Hong Kong',
 'Iran',
 'Laos',
 'Russia',
 'South Korea',
 'Turkey',
 'Venezuela',
 'Vietnam',
 'Yemen',
 nan}

In [17]:
set(merged['country']) - set(lane_and_ferretti['country'])

{'Africa Eastern and Southern',
 'Africa Western and Central',
 'American Samoa',
 'Arab World',
 'Armenia',
 'Cabo Verde',
 'Caribbean small states',
 'Central African Republic',
 'Central Europe and the Baltics',
 'Channel Islands',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 "Cote d'Ivoire",
 'Cuba',
 'Curacao',
 'Czechia',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (IDA & IBRD countries)',
 'East Asia & Pacific (excluding high income)',
 'Egypt, Arab Rep.',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Europe & Central Asia (excluding high income)',
 'European Union',
 'Fragile and conflict affected situations',
 'Greenland',
 'Guam',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'Hong Kong SAR, China',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Iran, Islamic Rep.',
 "Korea, Dem. People's Rep.",
 'Korea, Rep.',
 'Lao PDR',
 'Late-demographic dividend',
 'Latin A

In [18]:
# Fix merged country names for last merge (This will prevent information loss):

# Values that are to be changed:
wb_old = ['Czechia', 'Egypt, Arab Rep.', 'Iran, Islamic Rep.',
              'Korea, Rep.', 'Lao PDR', 'Russian Federation',
              'Turkiye', 'Venezuela, RB', 'Viet Nam',
              'Yemen, Rep.']

# Replace values:
wb_new = ['Czech Republic', 'Egypt', 'Iran',
          'Korea', 'Laos', 'Russia',
          'Turkey', 'Venezuela', 'Vietnam',
          'Yemen']

In [19]:
# Replace the values:
for old_value, new_value in zip(wb_old, wb_new):
    
    # Find and replace old valies with new values in merged:
    merged['country'].replace(old_value, new_value, inplace=True)

In [20]:
# Merge datasets:
merged = pd.merge(left=merged, right=lane_and_ferretti[['country', 'year', 'FINANCIAL']],
                  how='inner', on=['country', 'year'])

In [21]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5157 entries, 0 to 5156
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    5157 non-null   object 
 1   code       5157 non-null   object 
 2   year       5157 non-null   object 
 3   n          5156 non-null   float64
 4   pop        5157 non-null   float64
 5   I          4171 non-null   float64
 6   y          4882 non-null   float64
 7   TRADE      4310 non-null   float64
 8   INST       4419 non-null   float64
 9   HC         3264 non-null   float64
 10  FINANCIAL  5097 non-null   float64
dtypes: float64(8), object(3)
memory usage: 443.3+ KB


In [22]:
"""Define some filters
if Population less than 1 million (at 1996) then drop these countries
"""

one_million = 1_000_000

# Make year column integer:
merged['year'] = merged['year'].astype(int)

# Detect counties whose population less than a million in 1996:
to_drop_codes = merged.query(f'year == 1996 & pop < {one_million}')['code'].to_list()
merged = merged.query('code not in @to_drop_codes')

In [23]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3861 entries, 0 to 5156
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    3861 non-null   object 
 1   code       3861 non-null   object 
 2   year       3861 non-null   int64  
 3   n          3861 non-null   float64
 4   pop        3861 non-null   float64
 5   I          3434 non-null   float64
 6   y          3759 non-null   float64
 7   TRADE      3505 non-null   float64
 8   INST       3418 non-null   float64
 9   HC         2952 non-null   float64
 10  FINANCIAL  3825 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 362.0+ KB


In [24]:
len(merged['country'].unique())

143

In [25]:
# Additonal country removals
additonal_remove = merged[merged['y'].isna() | merged['I'].isna()]['code'].unique()
merged = merged.query('code not in @additonal_remove')

In [26]:
len(merged['country'].unique())

109

In [27]:
additonal_remove = merged[merged['TRADE'].isna()]['code'].unique()
merged = merged.query('code not in @additonal_remove')

In [28]:
len(merged['country'].unique())

107

In [29]:
additonal_remove = merged[merged['FINANCIAL'].isna()]['code'].unique()
merged = merged.query('code not in @additonal_remove')

In [30]:
len(merged['country'].unique())

106

In [31]:
additonal_remove = merged.query('year < 2020')[merged.query('year < 2020')['HC'].isna()]['code'].unique()
merged = merged.query('code not in @additonal_remove')

In [32]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2646 entries, 54 to 5156
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    2646 non-null   object 
 1   code       2646 non-null   object 
 2   year       2646 non-null   int64  
 3   n          2646 non-null   float64
 4   pop        2646 non-null   float64
 5   I          2646 non-null   float64
 6   y          2646 non-null   float64
 7   TRADE      2646 non-null   float64
 8   INST       2352 non-null   float64
 9   HC         2352 non-null   float64
 10  FINANCIAL  2646 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 248.1+ KB


In [33]:
# Generate OECD (for pre 1997) data:
oecd_pre_1997 = [
    "Australia",
    "Austria",
    "Belgium",
    "Canada",
    "Denmark",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Iceland",
    "Ireland",
    "Italy",
    "Japan",
    "Luxembourg",
    "Netherlands",
    "New Zealand",
    "Norway",
    "Portugal",
    "Spain",
    "Sweden",
    "Switzerland",
    "Turkey",
    "United Kingdom",
    "United States"
]

# Validate country names:
for country in oecd_pre_1997:
    if len(merged.query('country == @country')) == 0:
        print(f'{country} is not found in the dataset.')

Iceland is not found in the dataset.
Luxembourg is not found in the dataset.


In [34]:
merged['OECD'] = merged['country'].apply(lambda country: 1 if country in oecd_pre_1997 else 0)

In [35]:
# Interpolate rule of law index data:
merged['INST'] =  merged['INST'].interpolate(method='linear')

In [36]:
# Extrapolate last 3 years data for human capital index using linear regession:
def linear_extrapolation(df: pd.DataFrame,
                         most_recent_available_year: int,
                         target_column: str,
                         target_interval: list) -> tuple:
    """Finds the linear relationship and extrapolates the missing data"""
    
    # Find the missing rows:
    missing_data_idx = df.query('year in @target_interval').index
    
    # Find the linear relationship:
    # Assuming 'Year' and 'Value' are columns representing the time and the value to be predicted
    X_train = df.query('year <= @most_recent_available_year')[['year']].tail(5)  # Features (years before the missing ones)
    y_train = df.query('year <= @most_recent_available_year')[[target_column]].tail(5)   # Target (corresponding values)

    model = LinearRegression(fit_intercept=True)
    model.fit(X_train, y_train)
    predicted_values = model.predict(np.array(target_interval).reshape(3,1)).reshape(3,).tolist()
    pairs = zip(missing_data_idx, predicted_values)
    
    return pairs

target_interval = [2020, 2021, 2022]
target_column = 'HC'

for country in merged['country'].unique():
    missing_data = merged.query('country == @country')    
    
    # Apply to all countries:
    temp = linear_extrapolation(df=missing_data,
                                most_recent_available_year=2019,
                                target_column=target_column,
                                target_interval=target_interval)

    for idx, value in temp:
        merged.at[idx, target_column] = value
    
    del temp

In [37]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2646 entries, 54 to 5156
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    2646 non-null   object 
 1   code       2646 non-null   object 
 2   year       2646 non-null   int64  
 3   n          2646 non-null   float64
 4   pop        2646 non-null   float64
 5   I          2646 non-null   float64
 6   y          2646 non-null   float64
 7   TRADE      2646 non-null   float64
 8   INST       2646 non-null   float64
 9   HC         2646 non-null   float64
 10  FINANCIAL  2646 non-null   float64
 11  OECD       2646 non-null   int64  
dtypes: float64(8), int64(2), object(2)
memory usage: 333.3+ KB


In [38]:
merged

,country,code,year,n,pop,I,y,TRADE,INST,HC,FINANCIAL,OECD
54,Algeria,DZA,1996,1.7633,28984634.0000,25.6784,2927.3774,53.7051,-1.2182,1.7472,91.5847,0
55,Algeria,DZA,1997,1.6812,29476031.0000,22.4470,2910.2392,52.2439,-1.2083,1.7807,93.6758,0
56,Algeria,DZA,1998,1.5106,29924668.0000,27.1485,3012.8054,45.0945,-1.1984,1.8148,91.6812,0
57,Algeria,DZA,1999,1.3984,30346083.0000,26.2477,3066.0377,50.9291,-1.2221,1.8496,80.1083,0
58,Algeria,DZA,2000,1.4023,30774621.0000,23.5639,3138.2300,62.8583,-1.2458,1.8851,81.1204,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5152,Zimbabwe,ZWE,2018,2.0205,15052184.0000,14.1483,1462.5903,54.5503,-1.2925,2.6806,49.2666,0
5153,Zimbabwe,ZWE,2019,1.9893,15354608.0000,13.7994,1342.9896,55.7960,-1.3035,2.7134,73.2515,0
5154,Zimbabwe,ZWE,2020,2.0311,15669666.0000,13.1461,1213.1171,47.3134,-1.3296,2.7452,85.6084,0
5155,Zimbabwe,ZWE,2021,2.0457,15993524.0000,15.5071,1289.1991,50.8471,-1.2772,2.7774,80.9890,0


In [39]:
# Export the finalized data:
merged.to_csv('data.csv', index=False)